In [1]:
import pandas as pd

In [2]:
articles = pd.read_csv("train_subset.csv")
articles.head()

,Unnamed: 0,article,hyperpartisan,bias,url,labeled-by
0,1,\tWhen explaining her decision to reevaluate T...,True,right,https://dailywire.com/news/20858/after-devos-a...,publisher
1,2,\tA Florida university will honor Trayvon Mart...,True,right,https://dailywire.com/news/16103/university-aw...,publisher
2,8,\tNov. 15 ( UPI ) — Texas State University has...,False,right-center,https://newsline.com/texas-state-university-su...,publisher
3,12,\tAgainst the backdrop of an increasingly pola...,True,right,https://dailywire.com/node/9485,publisher
4,15,\tPresident-elect Donald Trump has reached an ...,True,right,https://dailywire.com/news/11184/breaking-trum...,publisher


In [3]:
articles["article"]

0        \tWhen explaining her decision to reevaluate T...
1        \tA Florida university will honor Trayvon Mart...
2        \tNov. 15 ( UPI ) — Texas State University has...
3        \tAgainst the backdrop of an increasingly pola...
4        \tPresident-elect Donald Trump has reached an ...
                               ...                        
10628    \tAug. 2 ( UPI ) — A possible U.S. sales deal ...
10629    \tThe Blessed Hope `` Looking for that blessed...
10630    \tAn exhibit that the artist says contains “ i...
10631    \tRepresentatives from the High HOPES ( Healin...
10632    \t... ... ... . ... ... ... . ... ... ... . .....
Name: article, Length: 10633, dtype: object

In [4]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
vectorizer = CountVectorizer(min_df=3, lowercase=False)
vectorizer.fit(articles["article"])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=False, max_df=1.0, max_features=None, min_df=3,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [6]:
vectorizer.vocabulary_

{'When': 18628,
 'explaining': 28363,
 'her': 31040,
 'decision': 25447,
 'to': 44914,
 'reevaluate': 39683,
 'Title': 17494,
 'IX': 9120,
 'guidelines': 30563,
 'as': 20516,
 'they': 44676,
 'pertain': 37373,
 'sexual': 41714,
 'assault': 20572,
 'on': 36219,
 'college': 23694,
 'campuses': 22551,
 'Secretary': 15829,
 'of': 36128,
 'Education': 6435,
 'Betsy': 3175,
 'DeVos': 5637,
 'said': 41030,
 'Every': 6808,
 'survivor': 44011,
 'misconduct': 34932,
 'must': 35409,
 'be': 21212,
 'taken': 44263,
 'seriously': 41643,
 'student': 43584,
 'accused': 19339,
 'know': 33246,
 'that': 44611,
 'guilt': 30569,
 'is': 32867,
 'not': 35878,
 'predetermined': 38177,
 'The': 17355,
 'Obama': 12924,
 'administration': 19487,
 'changes': 23006,
 'have': 30861,
 'been': 21273,
 'criticized': 24995,
 'for': 29358,
 'among': 19996,
 'other': 36439,
 'things': 44691,
 'substantially': 43707,
 'lowering': 34035,
 'the': 44618,
 'burden': 22326,
 'proof': 38638,
 'it': 32891,
 'pertains': 37375,
 'w

In [31]:
vectIn = vectorizer.transform(articles["article"])

In [32]:
from sklearn.feature_selection import SelectKBest, chi2
X_train = SelectKBest(chi2, k=10000).fit_transform(vectIn, articles["hyperpartisan"])
Y_train = articles["hyperpartisan"]

In [33]:
num_features_train = X_train.shape[1]
num_features_train

10000

In [34]:
#svm1 = sklearn.svm.SVC(gamma = 'auto', kernel='linear', cache_size=1000, C=1)
#svm1.fit(X_train[500:], articles["hyperpartisan"][500:])

In [35]:
#from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score
#print(accuracy_score(svm1.predict(X_train[:500]), articles["hyperpartisan"][:500]))

In [36]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(articles["hyperpartisan"])
encoded_Y = encoder.transform(articles["hyperpartisan"])

In [37]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import optimizers

In [38]:
model = Sequential() # This will be our neural network. Sequential means that the output of each layer is the input for the next one.
model.add(Dense(16, input_dim=num_features_train, activation='relu')) # Our network has as input the same number of features of the training set (i.e. 20 in our case).
#model.add(Dropout(0.25))
#model.add(Dense(32, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(8, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid')) # The output layer has in this case the same number of dimensions as our label one-hot vectors (i.e. 4 in our case).
print (model.summary()) # .summary shows an overview of our model

#adam = keras.optimizers.Adam(learning_rate=0.1, beta_1=0.9, beta_2=0.999, amsgrad=False)
#sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy']) # This serves to define the loss function and optimizer.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 16)                160016    
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 17        
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
_________________________________________________________________
None


In [39]:
history = model.fit(X_train, Y_train, validation_split=0.1, epochs=15, batch_size=64)

Train on 9569 samples, validate on 1064 samples
Epoch 1/15
9569/9569 [==============================] - 3s 357us/step - loss: 0.4733 - acc: 0.7897 - val_loss: 0.3427 - val_acc: 0.8703
Epoch 2/15
9569/9569 [==============================] - 3s 287us/step - loss: 0.2651 - acc: 0.9041 - val_loss: 0.2611 - val_acc: 0.8976
Epoch 3/15
9569/9569 [==============================] - 3s 305us/step - loss: 0.1734 - acc: 0.9409 - val_loss: 0.2425 - val_acc: 0.9145
Epoch 4/15
9569/9569 [==============================] - 2s 249us/step - loss: 0.1315 - acc: 0.9599 - val_loss: 0.2495 - val_acc: 0.9126
Epoch 5/15
9569/9569 [==============================] - 3s 277us/step - loss: 0.0976 - acc: 0.9729 - val_loss: 0.2475 - val_acc: 0.9145
Epoch 6/15
9569/9569 [==============================] - 2s 248us/step - loss: 0.0722 - acc: 0.9821 - val_loss: 0.2385 - val_acc: 0.9098
Epoch 7/15
9569/9569 [==============================] - 3s 283us/step - loss: 0.0544 - acc: 0.9889 - val_loss: 0.2575 - val_acc: 0.9135


In [40]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

In [41]:
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=num_features_train, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [43]:
for i in range(15): 
    print(model.predict(X_train[i]))

[[0.9537471]]
[[0.94431436]]
[[0.0134334]]
[[0.99569196]]
[[0.999897]]
[[4.1858337e-05]]
[[0.00017084]]
[[0.00240672]]
[[2.6112932e-06]]
[[0.00182227]]
[[0.9997373]]
[[0.9999995]]
[[0.00010746]]
[[0.01634045]]
[[0.00447841]]
